# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,atafu village,-8.5421,-172.5159,28.63,76,64,2.82,TK,1698446300
1,1,port mathurin,-19.6833,63.4167,24.43,76,96,3.28,MU,1698446569
2,2,puerto ayora,-0.7393,-90.3518,27.98,88,50,4.47,EC,1698446569
3,3,st. john's,47.5649,-52.7093,6.05,85,40,2.06,CA,1698446570
4,4,tolanaro,-25.0319,46.9987,20.90,85,72,5.49,MG,1698446570


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_city_df = city_data_df.loc[
    (city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21) \
    & (city_data_df["Wind Speed"] < 4.5) \
    & (city_data_df["Cloudiness"] == 0)
]

# Drop any rows with null values
narrowed_city_df = narrowed_city_df.dropna()

# Display sample data
narrowed_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
10,10,hafar al-batin,28.4328,45.9708,21.32,35,0,0.00,SA,1698446572
59,59,westport,41.1415,-73.3579,21.63,74,0,2.06,US,1698446584
85,85,shushtar,32.0455,48.8567,23.31,35,0,1.50,IR,1698446591
89,89,fort bragg,35.1390,-79.0060,25.32,64,0,2.57,US,1698446592
98,98,morondava,-20.2833,44.2833,22.87,75,0,2.18,MG,1698446595
128,128,mponela,-13.5319,33.7401,21.73,55,0,2.22,MW,1698446603
136,136,hassi messaoud,31.6804,6.0729,21.92,26,0,1.54,DZ,1698446605
137,137,awjilah,29.1081,21.2869,21.89,55,0,2.69,LY,1698446605
275,275,zouerate,22.7187,-12.4521,25.41,30,0,3.95,MR,1698446646
303,303,bechar,31.6167,-2.2167,22.18,21,0,1.03,DZ,1698446654


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrowed_city_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
10,hafar al-batin,SA,28.4328,45.9708,35,
59,westport,US,41.1415,-73.3579,74,
85,shushtar,IR,32.0455,48.8567,35,
89,fort bragg,US,35.1390,-79.0060,64,
98,morondava,MG,-20.2833,44.2833,75,
128,mponela,MW,-13.5319,33.7401,55,
136,hassi messaoud,DZ,31.6804,6.0729,26,
137,awjilah,LY,29.1081,21.2869,55,
275,zouerate,MR,22.7187,-12.4521,30,
303,bechar,DZ,31.6167,-2.2167,21,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
hafar al-batin - nearest hotel: فندق الفرحان
westport - nearest hotel: Norwalk Inn and Conference Center
shushtar - nearest hotel: هتل سنتی سرابی شوشتر
fort bragg - nearest hotel: Airborne Inn Lodging
morondava - nearest hotel: Hotel Sharon Menabe
mponela - nearest hotel: Centurion Guest House
hassi messaoud - nearest hotel: AMC
awjilah - nearest hotel: No hotel found
zouerate - nearest hotel: فندق تيرس زمور
bechar - nearest hotel: Hôtel Madania
laguna - nearest hotel: Holiday Inn Express & Suites
rosarito - nearest hotel: WorldMark La Paloma
douz - nearest hotel: Louage station Douz
saint-pierre - nearest hotel: Tropic Hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
10,hafar al-batin,SA,28.4328,45.9708,35,فندق الفرحان
59,westport,US,41.1415,-73.3579,74,Norwalk Inn and Conference Center
85,shushtar,IR,32.0455,48.8567,35,هتل سنتی سرابی شوشتر
89,fort bragg,US,35.1390,-79.0060,64,Airborne Inn Lodging
98,morondava,MG,-20.2833,44.2833,75,Hotel Sharon Menabe
128,mponela,MW,-13.5319,33.7401,55,Centurion Guest House
136,hassi messaoud,DZ,31.6804,6.0729,26,AMC
137,awjilah,LY,29.1081,21.2869,55,No hotel found
275,zouerate,MR,22.7187,-12.4521,30,فندق تيرس زمور
303,bechar,DZ,31.6167,-2.2167,21,Hôtel Madania


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
"Lng",
"Lat",
geo = True,
size = "Humidity",
scale = 1,
color = "City",
alpha = 0.5,
tiles = "OSM",
frame_width = 700,
frame_height = 500,
hover_cols = ["Hotel Name", "Country"]
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)